<a href="https://colab.research.google.com/github/Coderified/NLP_Repo/blob/main/NLP_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC

from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Activation, Dropout, Embedding
from keras.layers import BatchNormalization
# from tensorflow.keras.utils import np_utils # np_utils is deprecated, use tf.keras.utils instead if needed
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence
# from tensorflow.keras.preprocessing.text import text # Corrected import path
from keras.callbacks import EarlyStopping

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import string

from nltk import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

In [42]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/spooky-author-identification/train/train.csv'
test_path = '/content/drive/MyDrive/spooky-author-identification/test/test.csv'
samplesub_path = '/content/drive/MyDrive/spooky-author-identification/sample_submission/sample_submission.csv'

train = pd.read_csv(file_path)
test = pd.read_csv(test_path)
ss = pd.read_csv(samplesub_path)
train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


Error Metric - Multi Class Log loss


In [30]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [43]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(train['author'])

In [44]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y,
                                                  stratify=y,
                                                  random_state=42,
                                                  test_size=0.1, shuffle=True)

Text values from train df is split and stored in xtrain and xval, and y is the label encoded values

In [49]:
xvalid.shape

(1958,)

In [53]:
tfv = TfidfVectorizer(min_df=3,  max_features=None,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=True,smooth_idf=True,sublinear_tf=True,stop_words = 'english')

tfv.fit(list(xtrain)+list(xvalid))
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

| Parameter                 | Description                                                                                                                  |
| ------------------------- | ---------------------------------------------------------------------------------------------------------------------------- |
| `min_df=3`                | **Ignore terms that appear in fewer than 3 documents**. Helps remove noise and rare terms.                                   |
| `max_features=None`       | No cap on the vocabulary size. If you'd used, say, `max_features=5000`, it would keep only the top 5,000 terms by frequency. |
| `strip_accents='unicode'` | Normalize unicode characters (e.g., convert é → e). Useful for multilingual or accented text.                                |
| `analyzer='word'`         | Tokenize text at the **word** level (default).                                                                               |
| `token_pattern=r'\w{1,}'` | A custom regex to define what a "word" is. `\w{1,}` means: any word with at least 1 character (letters/numbers/underscores). |
| `ngram_range=(1, 3)`      | Extract **unigrams, bigrams, and trigrams**. So you'll get tokens like: "hello", "hello world", "hello world again".         |
| `use_idf=True`            | Use **inverse document frequency** — core part of TF-IDF. Penalizes terms that appear in many documents.                     |
| `smooth_idf=True`         | Adds 1 to document frequencies to **avoid divide-by-zero errors** when computing IDF.                                        |
| `sublinear_tf=True`       | Applies **logarithmic scaling to term frequency**: `tf = 1 + log(tf)` instead of raw counts. Helps with very frequent words. |
| `stop_words='english'`    | Removes common English stopwords like "the", "and", "is", etc., to reduce noise.                                             |


In [56]:
xtrain_tfv

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 198521 stored elements and shape (17621, 15102)>

In [59]:
tfv.get_feature_names_out()[0:50] # show first 10

array(['abandon', 'abandoned', 'abandoning', 'abandonment', 'abaout',
       'abbey', 'abdication', 'abdul', 'abdul alhazred', 'abernethy',
       'abeyance', 'abhor', 'abhorred', 'abhorrence', 'abhorrent',
       'abilities', 'ability', 'abject', 'able', 'abnormal',
       'abnormalities', 'abnormality', 'abnormally', 'abode', 'abodes',
       'abominable', 'abroad', 'abrupt', 'abruptly', 'abruptness',
       'absence', 'absence defects', 'absence defects incongruities',
       'absence perdita', 'absent', 'absolute', 'absolutely',
       'absolutely necessary', 'absorbed', 'absorbing', 'abstract',
       'abstracted', 'abstraction', 'abstruse', 'absurd', 'absurdities',
       'absurdity', 'absurdly', 'abundance', 'abundant'], dtype=object)

In [68]:
lr = LogisticRegression(C=1, solver='liblinear')
lr.fit(xtrain_tfv, ytrain)
preds = lr.predict_proba(xvalid_tfv) # Use predict_proba to get probabilities since logloss expects the probs in 3 column format here
print("LogLoss : ",multiclass_logloss(yvalid,preds))

LogLoss :  0.6260093644831317


In [69]:
preds

array([[0.64570316, 0.07198908, 0.28230776],
       [0.7279392 , 0.11053272, 0.16152808],
       [0.59463897, 0.16063728, 0.24472375],
       ...,
       [0.30960304, 0.23045395, 0.45994301],
       [0.27844526, 0.19130916, 0.53024558],
       [0.12123526, 0.80918181, 0.06958293]])

Count Venctorizer

In [70]:
ctv = CountVectorizer(analyzer = 'word', token_pattern = r'\w{1,}', ngram_range = (1,3), stop_words = 'english')
ctv.fit(list(xtrain)+list(xvalid))
xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

clf = LogisticRegression(C=1, solver='liblinear')
clf.fit(xtrain_ctv, ytrain)
preds = clf.predict_proba(xvalid_ctv)
print("LogLoss : ",multiclass_logloss(yvalid,preds))

LogLoss :  0.5283148537165375


In [71]:
mnb_tfv = MultinomialNB()
mnb_tfv.fit(xtrain_tfv, ytrain)
preds = mnb_tfv.predict_proba(xvalid_tfv)
print("LogLoss - NaiveBayes - TFIDF : ",multiclass_logloss(yvalid,preds))

mnb_ctv = MultinomialNB()
mnb_ctv.fit(xtrain_ctv, ytrain)
preds = mnb_ctv.predict_proba(xvalid_ctv)
print("LogLoss - NaiveBayes - CV : ",multiclass_logloss(yvalid,preds))

LogLoss - NaiveBayes - TFIDF :  0.5778049688756707
LogLoss - NaiveBayes - CV :  0.48541492313489554


In [72]:
from xgboost import XGBClassifier

In [73]:
print(type(xtrain_tfv))

<class 'scipy.sparse._csr.csr_matrix'>


✅ Summary
.tocsc() Does What?	Converts sparse matrix to CSC (column-based format)
Why needed here?	Optimizes XGBoost’s performance, especially with text features
When to use it?	When input is large, sparse, and especially when using TF-IDF or similar vectorizers

In [74]:
xgb_tfv = XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
subsample=0.8, nthread=10, learning_rate=0.1)
xgb_tfv.fit(xtrain_tfv.tocsc(), ytrain)
preds = xgb_tfv.predict_proba(xvalid_tfv.tocsc())
print("LogLoss - XGBoost - TFIDF : ",multiclass_logloss(yvalid,preds))

xgb_ctv = XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,
subsample=0.8, nthread=10, learning_rate=0.1)
xgb_ctv.fit(xtrain_ctv.tocsc(), ytrain)
preds = xgb_ctv.predict_proba(xvalid_ctv.tocsc())
print("LogLoss - XGBoost - CV : ",multiclass_logloss(yvalid,preds))

LogLoss - XGBoost - TFIDF :  0.7833459138127274
LogLoss - XGBoost - CV :  0.7731896190451263


# WORD VECTOR
GLOVE

🧠 What is GloVe?
GloVe (Global Vectors for Word Representation) is a pretrained word embedding method developed by Stanford NLP. It’s a way to represent words as dense vectors (typically 50 to 300 dimensions), where semantic relationships between words are captured mathematically.

Unlike TF-IDF or Bag-of-Words (which are sparse and high-dimensional), GloVe embeddings are:

Dense (e.g., 100-dimensional float vectors)

Context-aware to some extent (but static per word)

Learned from co-occurrence statistics over large corpora like Wikipedia or Common Crawl

🧬 How is GloVe Trained?
GloVe is trained on a word co-occurrence matrix from a massive corpus.

If the word “king” often appears near “queen”, “royal”, and “crown”, their vectors will be close in space.

GloVe uses this idea to generate embeddings that:

Capture similarity (e.g., man ≈ male)

Capture analogies:

vec("king") - vec("man") + vec("woman") ≈ vec("queen")

📦 GloVe Vector Format
Each line in a GloVe file looks like this:

arduino
Copy
Edit
word 0.418 0.24968 -0.41242 ... 0.1234
The first token is the word

The rest are the vector components

In [75]:
# Load GloVe embeddings
embedding_index = {}

with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'